In [ ]:
import cv2
from ultralytics import YOLO
import supervision as sv
import numpy as np

# Define the zone polygon
ZONE_POLYGON = np.array([
    [0,0],
    [1280,0],
    [1280,370],
    [0,370]
])

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()
  
model = YOLO('./yolov8n.pt')

bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

zone = sv.PolygonZone(polygon=ZONE_POLYGON)
zone_annotator = sv.PolygonZoneAnnotator(zone=zone, color=sv.Color(255,0,0))

while True:
    ret, frame = cap.read()
    if not ret:
        print("Image capture failed")
        break

    result = model(frame, agnostic_nms=True)[0]
    detections = sv.Detections.from_ultralytics(result)

    # List to store detected cards in the zone
    d_cards_detected = []
    p_cards_detected = []

    # Process detections
    for i in range(len(detections)):
        xyxy_tensor = detections[i].xyxy
        xyxy = xyxy_tensor.squeeze()
        xmin, ymin, xmax, ymax = xyxy.astype(int)

        classidx = int(detections[i].class_id)
        classname = model.model.names[classidx]
        conf = detections[i].confidence.item()

        if zone.trigger(detections[i])[0]:
            d_cards_detected.append(classname)
        else :
            p_cards_detected.append(classname)
            
   
    if len(detections) > 0:
        labels = [
            f"{model.model.names[class_id]} {confidence:.2f}"
            for class_id, confidence in zip(detections.class_id, detections.confidence)
        ]
    else:
        labels = []

    annotated_frame = bounding_box_annotator.annotate(
        scene=frame, detections=detections
    )
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame, detections=detections, labels=labels
    )


    annotated_frame = zone_annotator.annotate(scene=annotated_frame)

     # Display detected cards in the zones
    cv2.rectangle(annotated_frame, (10, 10), (300, 100), (50, 50, 50), cv2.FILLED)
    cv2.putText(annotated_frame, f'Dealer Cards: {", ".join(d_cards_detected)}', (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.rectangle(annotated_frame, (10, 390), (300, 480), (50, 50, 50), cv2.FILLED)
    cv2.putText(annotated_frame, f'Player Cards: {", ".join(p_cards_detected)}', (20, 430), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)


    cv2.imshow("detection", annotated_frame)


    key_press = cv2.waitKey(5)
    if key_press == ord('q'):
        break
    elif key_press == ord('p'):
        cv2.imwrite('table.png', annotated_frame)

cap.release()
cv2.destroyAllWindows()


0: 480x640 (no detections), 37.7ms
Speed: 1.8ms preprocess, 37.7ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 40.5ms
Speed: 1.3ms preprocess, 40.5ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 39.3ms
Speed: 1.2ms preprocess, 39.3ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 30.4ms
Speed: 1.1ms preprocess, 30.4ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 40.9ms
Speed: 1.2ms preprocess, 40.9ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 36.1ms
Speed: 1.0ms preprocess, 36.1ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 38.6ms
Speed: 1.4ms preprocess, 38.6ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 41.0ms
Speed: 1.1ms preprocess, 41.0ms i


0: 480x640 1 10D, 32.7ms
Speed: 1.0ms preprocess, 32.7ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 8C, 1 10H, 29.4ms
Speed: 1.1ms preprocess, 29.4ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 32.6ms
Speed: 1.2ms preprocess, 32.6ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 31.2ms
Speed: 1.8ms preprocess, 31.2ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 31.1ms
Speed: 0.9ms preprocess, 31.1ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 27.9ms
Speed: 0.9ms preprocess, 27.9ms inference, 0.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 10H, 32.6ms
Speed: 1.1ms preprocess, 32.6ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 32.6ms
Speed: 0.9ms preprocess, 32.6ms inference, 0.2ms postproc